In [24]:
!pip install geopandas
!pip install arcgis

In [0]:
import pandas as pd
import geopandas as gpd
from google.colab import files
from datetime import date
import folium
import branca.colormap as cm
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame

In [26]:
# State DHHS Dataframe
item = GIS().content.get("969678bce431494a8f64d7faade6e5b8")
flayer = item.layers[0]
sdf = SpatialDataFrame.from_layer(flayer)
sdf.head()

/usr/local/lib/python3.6/dist-packages/arcgis/features/feature.py:612: UserWarning: The SpatialDataFrame has been deprecated. `df` property will be removed as a future release. Use `sdf` instead.
  warnings.warn(("The SpatialDataFrame has been deprecated. "
/usr/local/lib/python3.6/dist-packages/arcgis/features/_data/geodataset/geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID_1,OBJECTID,AREA,PERIMETER,ACRES,COCAPS,STFIPS,FIPSNUM,FIPS,CONAME,County,Total,Deaths,StateCases,USCases,USDeaths,CountyCases,Shape__Area,Shape__Length,SHAPE
0,1,1,0,156472.984375,366053.062500,ROCKINGHAM,37157,157,157,Rockingham,Rockingham,0,0,54,15219,201,35,0.148786,1.605509,"{'rings': [[[-79.532470223, 36.241411973], [-7..."
1,2,2,0,166183.500000,420968.937500,GUILFORD,37081,81,081,Guilford,Guilford,7,0,54,15219,201,35,0.170461,1.687631,"{'rings': [[[-80.035153615, 36.2571685120001],..."
2,3,3,0,143114.546875,278127.031250,ALAMANCE,37001,1,001,Alamance,Alamance,1,0,54,15219,201,35,0.112591,1.403250,"{'rings': [[[-79.532470223, 36.241411973], [-7..."
3,4,4,0,116938.171875,168640.984375,ALEXANDER,37003,3,003,Alexander,Alexander,0,0,54,15219,201,35,0.068157,1.159023,"{'rings': [[[-81.029146985, 36.045731872], [-8..."
4,5,5,0,180894.171875,505667.968750,RANDOLPH,37151,151,151,Randolph,Randolph,0,0,54,15219,201,35,0.203851,1.818204,"{'rings': [[[-80.0471221179999, 35.92091653000..."


In [27]:
# Get geometry of counties
nc_counties = gpd.read_file("https://opendata.arcgis.com/datasets/34acbf4a26784f189c9528c1cf317193_0.geojson")

# Get dataset with historical values
nc_timeline = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQn6xaVpOcC0TpFaH_BkQy0Av1ZJDeFfziD6iYR6sC-kiMSMz65J6uo9rh8dIS_sTQdh1RPFp9QMDBo/pub?gid=0&single=true&output=csv')
nc_timeline['ID'] = nc_timeline['ID'].str.upper()

# Combine datasets
df = nc_counties.merge(nc_timeline, left_on = 'CO_NAME', right_on = 'ID')

# Get current date's column position
date_col = df.columns.get_loc(str(date.today().strftime("%m/%d/%Y")))

# print(date_col)
df.head()

,FID,OBJECTID,PERIMETER,CO_NAME,CO_ABBR,ACRES,Shape_Leng,Shape_Area,RO,geometry,Unnamed: 0,ID,Sum,03/13/2020,03/14/2020,03/15/2020,03/16/2020,03/17/2020,03/18/2020,03/19/2020,03/20/2020,03/21/2020,03/22/2020,03/23/2020,03/24/2020,03/25/2020,03/26/2020,03/27/2020,03/28/2020,03/29/2020,03/30/2020,03/31/2020,04/01/2020,04/02/2020,04/03/2020,04/04/2020,04/05/2020,04/06/2020,04/07/2020,04/08/2020,04/09/2020,04/10/2020,04/11/2020,04/12/2020,04/13/2020
0,1,1,156472.984375,ROCKINGHAM,ROCK,366053.062500,513361.787377,1.594526e+10,WSRO,"POLYGON ((-79.53247 36.24141, -79.53201 36.250...",5,ROCKINGHAM,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,166183.500000,GUILFORD,GUIL,420968.937500,545220.367738,1.833739e+10,WSRO,"POLYGON ((-80.03515 36.25717, -80.03692 36.205...",9,GUILFORD,4,0,0,0,0,1,1,1,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,143114.546875,ALAMANCE,ALAM,278127.031250,469534.986553,1.211521e+10,WSRO,"POLYGON ((-79.53247 36.24141, -79.53299 36.231...",17,ALAMANCE,1,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,116938.171875,ALEXANDER,ALEX,168640.984375,383654.655146,7.345997e+09,MRO,"POLYGON ((-81.02915 36.04573, -81.04811 36.041...",2,ALEXANDER,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,180894.171875,RANDOLPH,RAND,505667.968750,593483.622166,2.202688e+10,WSRO,"POLYGON ((-80.04712 35.92092, -80.04887 35.880...",8,RANDOLPH,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Create map
m = folium.Map(
    location=[df.geometry.centroid.y.mean(), df.geometry.centroid.x.mean()], zoom_start=7, tiles=None)
folium.TileLayer(tiles='openstreetmap', show=True, control=False, min_zoom=5).add_to(m)

# Define today and search values
today = date.today().strftime("%m/%d/%Y")
search_value = today
search_label = 'Confirmed Cases (' + str(today) +")"

# print(colormap(5.0))

# Set color map dimensions
colormap = cm.LinearColormap(colors=['white','pink','red'], index=[df[search_value].min(),df[search_value].min()+4,df[search_value].max()],vmin=df[search_value].min(),vmax=df[search_value].max())
colormap

fg = folium.FeatureGroup(name=search_label, show=True)

# Sums for entire state
df['State Totals'] = df[search_value].sum()

style_function=lambda x: {
  'fillColor': colormap(x['properties'][today]),
  'color': 'black',
  'weight': 1,
  'fillOpacity': 0.45,
  'weight': 1,
  'opacity':0.4,
  'nan_fill_color': 'purple'
  }

folium.GeoJson(df, 
    style_function=style_function,

    tooltip=folium.GeoJsonTooltip(fields=['ID',search_value,'State Totals'],
        aliases=['<div style="background-color: white; color: black; padding: 3px; border: 0px solid black; border-radius: 5px;">'+item+'</div>' for item in ['County','Confirmed Cases','Total NC Cases on ' + today]],
        style="font-family: Helvetica, Arial, sans-serif;",
        localize=True),
        highlight_function=lambda x: {"weight":3,'fillOpacity':0.8}
      ).add_to(fg)

fg.add_to(m)
colormap.caption = search_label
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

m

# Exports maps just for current date
filename = date.today().strftime("%Y%m%d")+ '_NC_COVID.html'
filename2 = 'CovidMap_Today.html'

from google.colab import files
m.save(filename)
m.save(filename2)
files.download(filename)
files.download(filename2)

In [0]:
# All In One Map
m = folium.Map(
    location=[df.geometry.centroid.y.mean(), df.geometry.centroid.x.mean()], zoom_start=7, tiles=None)

folium.TileLayer(tiles='openstreetmap', show=True, control=False, min_zoom=5).add_to(m)

#https://nbviewer.jupyter.org/gist/jtbaker/c9b94010a160bd2120f5b3dde082ddf1

# today = date.today().strftime("%m/%d/%Y")

# print(colormap(5.0))

date_columns = sorted(set(df.columns[13:date_col+1]))
color_multiplier = 1
color_phases = len(date_columns)

def map_maker(date):  
  search_value = date
  colormap = cm.LinearColormap(colors=['white','pink','red'], index=[df[search_value].min(),df[search_value].max()*0.35,df[search_value].max()],vmin=df[search_value].min(),vmax=df[search_value].max()) #.to_step(int(color_multiplier*3))
  df['State Totals'] = df[search_value].sum()
  
  # colormap
  search_label = 'Confirmed Cases'
  fg = folium.FeatureGroup(name=str(date), show=True) #+ ": "+search_label
  # search_value = 'Value'

  style_function=lambda x: {
    'fillColor': colormap(x['properties'][search_value]),
    'color': 'black',
    'weight': 1,
    'fillOpacity': 0.15 * (color_multiplier/color_phases),
    'weight': 1,
    'opacity':0.2,
    'nan_fill_color': 'purple',
    }

  folium.GeoJson(df, 
      style_function=style_function,
      tooltip=folium.GeoJsonTooltip(fields=['ID',search_value,'State Totals'],
        aliases=['<div style="background-color: white; color: black; padding: 3px; border: 0px solid black; border-radius: 5px;">'+item+'</div>' for item in ['County','Confirmed Cases','Total NC Cases on ' + date]],
        style="font-family: Helvetica, Arial, sans-serif;",
        localize=True),
        highlight_function=lambda x: {"weight":3,'fillOpacity':0.8}
      ).add_to(fg)
  fg.add_to(m)
# colormap.caption = "Confirmed cases on " + str(date_columns[-1:])
# colormap.add_to(m)  


for date in date_columns:
  map_maker(date)
  color_multiplier += 1

folium.LayerControl(collapsed=False).add_to(m)

# fs = plugins.Fullscreen().add_to(m)
m

filename = 'NC_Timeline.html'
from google.colab import files
m.save(filename)
files.download(filename)